# Fundamentos de BD (Márcio Victorino)

## Partes do Projeto:
1. Introdução.
1. Modelo de dados Relacional.
1. O script SQL que gerou o banco de dados.
1. Apresentar o processo de ETL (Extract, Transform, Load) para importação dos
dados para o banco de dados.
1. Utilização de pelo menos uma View.
1. Utilização de pelo menos uma Procedure (com comandos condicionais).
1. Utilização de pelo menos um trigger (com comandos condicionais).
1. No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

**Data Limite para a Entrega da Parte Escrita: 02/06/2023**

## Observações

- Dados abertos (ex: FNDE, bolsa família)
- Linkar com o Artigo da matéria do Ladeira MD
- Joga em um CSV e alimenta o BD SQL
- No mínimo milhares de dezenas de linhas
- Não executar a consulta na apresentação
- Printar os resultados e mostrar na apresentação

## Alunos: 

Marcelo Anselmo de Souza Filho
- Matrícula: **231109719**
- Email: **marcelofilho@mpf.mp.br**

Arivaldo Gonçalves de Freitas Junior
- Matrícula: **231109620**
- Email: **arivaldofreitas@correios.com.br**

Luciana Maria de Araujo Freitas
- Matrícula: **231109700**
- Email: **luciana@mpdft.mp.br**

____

# 1. Introdução

**Sobre**: Este estudo aborda a diferença salarial entre homens e mulheres na área de TI durante a pandemia. Ele também explora possíveis cenários para analisar a disparidade salarial e de desligamento entre gêneros na área de tecnologia, antes e após a pandemia. Utilizou-se dados a nível do indivíduo, de 2018 e 2019 (antes da pandamiea) e de 2020 e 2021 (durante a pandemia), obtidos da Relação Anual de Informações Sociais (Rais), que proporciona dados oficiais sobre o mercado de trabalho no Brasil. 

**Resultados**: No geral, constatou-se a remuneração média das mulheres é menor que a dos homens em todas as regiões do Brasil, tanto em 2019 quanto em 2020. Além disso, a quantidade de desligamento de homens é maior que a de mulheres em todas as regiões do Brasil, tanto em 2019 quanto em 2020.

# 2. Modelo de dados Relacional

## 2.1 Modelo Conceitual

https://mermaid.js.org/syntax/entityRelationshipDiagram.html

Integrando jupyter com mermaid: https://mermaid.js.org/config/Tutorials.html#jupyter-integration-with-mermaid-js

In [75]:
```mermaid
---
title: Modelo Entidade Relacionamento
---
erDiagram
    EMPREGADO {
        int id PK
        int ano "Ano que foi empregado"
        float remuneracao_media "Ex: 3000"
        boolean desligamento "Se foi demitido"
        int idade "Ex: 30"
        string ocupacao "Ex: Analista de Redes"
        string sexo "Ex: Masculino"
    }
    LOCAL_EMPRESA {
        int id PK
        string sigla_uf "Ex: DF"
        string regiao "Ex: Nordeste"
    }
    EMPREGADO ||--|{ LOCAL_EMPRESA : trabalha
    
```

<img src="querys_sql/prints/modelo-er.PNG"  />

## 2.2 Modelo Lógico

### 2.2.1 Normalização

_1º FN: Uma relação está em 1FN se e somente se todos os seus atributos contêm apenas valores atômicos (simples, indivisíveis)_

**RESPOSTA**: A relação está na 1FN, pois todos os atributos são simples e indivisíveis.

_2º FN: Uma relação encontra-se na 2FN se e somente se estiver em 1FN e não contém dependências parciais._

```mermaid
---
title: Modelo Lógico
---
erDiagram
    EMPREGADO {
import base64
from IPython.display import Image, display


def mm(graph):
  graph = graph.replace("\n", " ")
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))



In [77]:
mm("""
erDiagram    
    RAIS ||--|| OCUPACAO : contem
    RAIS ||--|| OCUPACAO : contem
    RAIS ||--|| SEXO : contem
    RAIS ||--|| UF : contem
    UF ||--|| REGIAO : faz_parte
    RAIS {
        int id PK
        int id_ocupacao FK
        int id_sexo FK
        int id_uf FK
        int ano
        float remuneracao_media
        int desligamento
        int idade
    }
    OCUPACAO {
        int id PK
        string nome
    }
    SEXO {
        int id PK
        string nome
    }
    UF {
        int id PK
        int id_regiao FK
        string sigla_uf
    }
    REGIAO {
        int id PK
        string nome
    }
    EMPREGADO ||--|{ OCUPACAO : trabalha
    EMPREGADO ||--|{ SEXO : contem
    EMPREGADO ||--|{ UF : situado
    UF ||--|{ REGIAO : agrupado
    
```


<img src="querys_sql/prints/modelo-fn.PNG"  />

_3º FN: Uma relação está em 3FN se e somente se estiver na 2FN e nenhum atributo não-primo (isto é, que não seja membro de uma chave) for transitivamente dependente da chave primária._

**RESPOSTA**: A relação está na 3FN, pois não há dependência transitiva.


### 2.2.2 Especificação do BD

- OCUPACAO (<ins>id</ins>, nome)

- SEXO (<ins>id</ins>, nome)

- REGIAO ( <ins>id</ins>, nome )

- UF (<ins>id</ins>, id_regiao, sigla_uf)
  - id_regiao **REFERENCIA** REGIAO(id)
    
- EMPREGADO (<ins>id</ins>, id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade)
  - id_ocupacao **REFERENCIA** OCUPACAO(id)
  - id_sexo **REFERENCIA** SEXO(id)
  - id_uf **REFERENCIA** UF(id)

## 2.3 Modelo Físico

```sql
OCUPACAO (
    id INT NOT NULL,
    nome VARCHAR(255) NOT NULL,
    PRIMARY KEY (id)
);

SEXO (
    id INT NOT NULL,
    nome VARCHAR(9) NOT NULL,
    PRIMARY KEY (id)
);

REGIAO (
    id INT NOT NULL,
    nome VARCHAR(12) NOT NULL,
    PRIMARY KEY (id)
);

UF (
    id INT NOT NULL,
    id_regiao INT NOT NULL,
    nome VARCHAR(2) NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (id_regiao) REFERENCES REGIAO(id)
);

EMPREGADO (
    id INT NOT NULL AUTO_INCREMENT,
    id_ocupacao INT NOT NULL,
    id_sexo INT NOT NULL,
    id_uf INT NOT NULL,
    id_ano INT NOT NULL,
    remuneracao_media FLOAT,
    desligamento INT,
    idade INT NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY (id_ocupacao) REFERENCES OCUPACAO(id),
    FOREIGN KEY (id_sexo) REFERENCES SEXO(id),
    FOREIGN KEY (id_ano) REFERENCES ANO(id),
    FOREIGN KEY (id_uf) REFERENCES UF(id)
);
```

# 3. O script SQL que gerou o banco de dados.

Criando o Banco de dados e as respectivas tabelas:
- Arquivo: [querys_sql/CRIAR_BD.sql](querys_sql/CRIAR_BD.sql)

# 4. Apresentar o processo de ETL (Extract, Transform, Load) para importação dos dados para o banco de dados.

Os passos do ETL estão contidos nos arquivos abaixo:

- Juntando os anos de 2018 a 2021 em um único arquivo.
  - Arquivo: [03_juntando_todos_anos.ipynb](../03_juntando_todos_anos.ipynb)

- Juntando os dados da RAIS com os dados de sexo e raça.
  - Arquivo: [04_fazendo_join_ocup_sexo_raca.ipynb](../04_fazendo_join_ocup_sexo_raca.ipynb)

## Resumindo os passos do ETL

### Extração

1. Após filtrar os dados da tabela Relação Anual de Informações Sociais ([RAIS](https://basedosdados.org/dataset/3e7c4d58-96ba-448e-b053-d385a829ef00?table=c3a5121e-f00d-41ff-b46f-bd26be8d4af3)) pelos anos de 2018 a 2021, foi feito o filtro pelos IDs de cargos de Tecnologia da informação conforme a Classificação Brasileira de Ocupações ([CBO](https://cbo.mte.gov.br/cbosite/pages/home.jsf)).
    - 212205: Engenheiro de Aplicativos em Computacao
    - 212210: Engenheiro de Equipamentos em Computacao
    - 212215: Engenheiros de Sistemas Operacionais em Computacao
  
    - 212305: Administrador de Banco de Dados
    - 212310: Administrador de Redes
    - 212315: Administrador de Sistemas Operacionais
    - 212320: Administrador em Segurança da Informação
  
    - 212405: Analista de Desenvolvimento de Sistemas
    - 212410: Analista de Redes e de Comunicacao de Dados
    - 212415: Analista de Sistemas de Automacao
    - 212420: Analista de Suporte Computacional
  
    - 317105: Programador de Internet
    - 317110: Programador de Sistemas de Informacao
    - 317115: Programador de Maquinas - Ferramenta com Comando Numerico
    - 317120: Programador de Multimidia

    - 317205: Operador de Computador (Inclusive Microcomputador)
    - 317210: Tecnico de Apoio ao Usuario de Informatica (Helpdesk)

1. Em seguida, obtivemos os dados dos profissionais de TI no Brasil entre os anos de 2018 a 2019 
- Quantidade **total**: 1.543.009
- Quantidade **por ano**:
  - 2021: 691.982
  - 2018: 466.852
  - 2020: 222.102
  - 2019: 162.073
 
### Transformação

1. Primeiro, juntamos os dados com a planilha de Sexo

        1,Masculino
        2,Feminino
        -1,Ignorado
       
1. Em seguida, juntamos os dados com a planilha com o nome dos Cargos

        212205,Engenheiro de Aplicativos em Computacao
        212210,Engenheiro de Equipamentos em Computacao
        ...
       
1. Logo após, selecionamos apenas a colunas necessárias e as renomeamos

1. Por fim, alteramos todos os dados com "idade" = 0 para o mínimo de 14 anos (que é o menor valor, retirando o zero)

### Carregamento

Fazendo INSERT dos dados das tabelas (todas menos a tabela EMPREGADO) no Banco:
- Arquivo: [querys_sql/CARGA_BD.sql](querys_sql/CARGA_BD.sql)

A tabela de EMPREGADO foi feita com o script python abaixo, pois continha mais de 1.5 milhões de registros

In [4]:
from __future__ import print_function
import pandas as pd
import mysql.connector
from datetime import date, datetime, timedelta


class CargaFullTabelaRAIS:
  def __init__(self, batch_size=1000, size_max=10000):
    self.batch_size = batch_size
    self.size_max = size_max
    self.cnx = mysql.connector.connect(
        user='root', password='root', database='projfbd')
    self.cursor = self.cnx.cursor()
    self.path_file_parquet = "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip"

  def carregar_CSV(self):
      # Mostrar mais colunas
    pd.set_option("display.max_columns", 100)
    pd.set_option('display.max_colwidth', 100)

    df = pd.read_parquet(self.path_file_parquet)

    qnt_total = len(df)

    print(f"""
      Quantidade de docs carregados: {qnt_total}
      """)
    return df

  def get_rais_from_pandas(self):
      df = self.carregar_CSV()
      counts = df[[
          "ocupacao_id", "sexo_id", "sigla_uf_id", "ano", "remuneracao_media", "desligamento", "idade"]]
      lista = counts.values.tolist()
      lista = [tuple(x) for x in lista]

      return lista

  def gerar_batch_insert_rais(self):
    print(f"""
      Inserindo os dados...
      """)

    rais_from_pandas = self.get_rais_from_pandas()

    for i in range(0, len(rais_from_pandas[:self.size_max]), self.batch_size):
      add_query = ("INSERT INTO EMPREGADO "
                   "(id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade) "
                   "VALUES ")

      vals = ", ".join((f"({str(id_ocupacao)}, {str(id_sexo)}, {str(id_uf)}, {str(ano)}, {str(remuneracao_media)}, {str(desligamento)}, {str(idade)})")
                       for id_ocupacao, id_sexo, id_uf, ano, remuneracao_media, desligamento, idade in rais_from_pandas[i:i+self.batch_size])
      print(f"\n\n -- ====  INSERT a partir do dado {i}")
      # print(add_query + vals)
      try:
        self.cursor.execute(add_query + vals)
      except Exception as e:
        print(e)
        print(i)

    # ============ COMMIT ============
    self.cnx.commit()

  def init(self):
    self.gerar_batch_insert_rais()

  def __del__(self):
    self.cursor.close()
    self.cnx.close()


# ================= INICIANDO

CargaFullTabelaRAIS(batch_size=500000, size_max=2000000).init()



      Inserindo os dados...
      

      Quantidade de docs carregados: 1543009
      


 -- ====  INSERT a partir do dado 0


 -- ====  INSERT a partir do dado 500000


 -- ====  INSERT a partir do dado 1000000


 -- ====  INSERT a partir do dado 1500000


# 5. Utilização de pelo menos uma View.

Criação da View "VW_RAIS_FULL" com JOIN da tabela EMPREGADO, como a central, e as demais tabelas conectadas a ela.
- Arquivo: [querys_sql/VIEW.sql](querys_sql/VIEW.sql)

<img src="querys_sql/prints/view.PNG"  />

# 6. Utilização de pelo menos uma Procedure (com comandos condicionais).

Conteúdo de: [querys_sql/PROCEDURE.sql](querys_sql/PROCEDURE.sql)

<img src="querys_sql/prints/procedure.PNG"  />

# 7. Utilização de pelo menos um trigger (com comandos condicionais).

Conteúdo de: [querys_sql/TRIGGER.sql](querys_sql/TRIGGER.sql)

<img src="querys_sql/prints/trigger.PNG"  />


# 8.  No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

Conteúdo de: [querys_sql/CONSULTAS_COMPLEXAS.sql](querys_sql/CONSULTAS_COMPLEXAS.sql)


- Consulta complexa 1

<img src="querys_sql/prints/consulta-complexa-1.PNG"  />

- Consulta complexa 2

<img src="querys_sql/prints/consulta-complexa-2.PNG"  />

- Consulta complexa 3

<img src="querys_sql/prints/consulta-complexa-3.PNG"  />

- Consulta complexa 4

<img src="querys_sql/prints/consulta-complexa-4.PNG"  />

- Consulta complexa 5

<img src="querys_sql/prints/consulta-complexa-5.PNG"  />